# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8f8c8a2a60>
├── 'a' --> tensor([[ 0.7180, -1.4251, -0.4023],
│                   [ 0.3873, -1.5008, -1.9809]])
└── 'x' --> <FastTreeValue 0x7f8f8c8a2d90>
    └── 'c' --> tensor([[ 3.9525, -0.4756, -0.1471, -1.0205],
                        [ 0.4388,  1.5708, -0.4817, -0.0550],
                        [-0.6711,  1.0422,  0.8273, -1.5164]])

In [4]:
t.a

tensor([[ 0.7180, -1.4251, -0.4023],
        [ 0.3873, -1.5008, -1.9809]])

In [5]:
%timeit t.a

93 ns ± 1.17 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8f8c8a2a60>
├── 'a' --> tensor([[-0.5460, -0.2399, -0.0844],
│                   [ 0.5589, -0.7021, -2.7327]])
└── 'x' --> <FastTreeValue 0x7f8f8c8a2d90>
    └── 'c' --> tensor([[ 3.9525, -0.4756, -0.1471, -1.0205],
                        [ 0.4388,  1.5708, -0.4817, -0.0550],
                        [-0.6711,  1.0422,  0.8273, -1.5164]])

In [7]:
%timeit t.a = new_value

76.4 ns ± 0.338 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7180, -1.4251, -0.4023],
               [ 0.3873, -1.5008, -1.9809]]),
    x: Batch(
           c: tensor([[ 3.9525, -0.4756, -0.1471, -1.0205],
                      [ 0.4388,  1.5708, -0.4817, -0.0550],
                      [-0.6711,  1.0422,  0.8273, -1.5164]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7180, -1.4251, -0.4023],
        [ 0.3873, -1.5008, -1.9809]])

In [11]:
%timeit b.a

76.4 ns ± 0.489 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2883, -1.5170, -1.0543],
               [-0.4292, -0.4423, -0.0597]]),
    x: Batch(
           c: tensor([[ 3.9525, -0.4756, -0.1471, -1.0205],
                      [ 0.4388,  1.5708, -0.4817, -0.0550],
                      [-0.6711,  1.0422,  0.8273, -1.5164]]),
       ),
)

In [13]:
%timeit b.a = new_value

673 ns ± 3.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.18 µs ± 14.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

15.1 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

291 µs ± 7.64 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

277 µs ± 8.67 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8ed4b64f40>
├── 'a' --> tensor([[[ 0.7180, -1.4251, -0.4023],
│                    [ 0.3873, -1.5008, -1.9809]],
│           
│                   [[ 0.7180, -1.4251, -0.4023],
│                    [ 0.3873, -1.5008, -1.9809]],
│           
│                   [[ 0.7180, -1.4251, -0.4023],
│                    [ 0.3873, -1.5008, -1.9809]],
│           
│                   [[ 0.7180, -1.4251, -0.4023],
│                    [ 0.3873, -1.5008, -1.9809]],
│           
│                   [[ 0.7180, -1.4251, -0.4023],
│                    [ 0.3873, -1.5008, -1.9809]],
│           
│                   [[ 0.7180, -1.4251, -0.4023],
│                    [ 0.3873, -1.5008, -1.9809]],
│           
│                   [[ 0.7180, -1.4251, -0.4023],
│                    [ 0.3873, -1.5008, -1.9809]],
│           
│                   [[ 0.7180, -1.4251, -0.4023],
│                    [ 0.3873, -1.5008, -1.9809]]])
└── 'x' --> <FastTreeValue 0x7f8f60d21460>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

48.6 µs ± 678 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8f8c899b20>
├── 'a' --> tensor([[ 0.7180, -1.4251, -0.4023],
│                   [ 0.3873, -1.5008, -1.9809],
│                   [ 0.7180, -1.4251, -0.4023],
│                   [ 0.3873, -1.5008, -1.9809],
│                   [ 0.7180, -1.4251, -0.4023],
│                   [ 0.3873, -1.5008, -1.9809],
│                   [ 0.7180, -1.4251, -0.4023],
│                   [ 0.3873, -1.5008, -1.9809],
│                   [ 0.7180, -1.4251, -0.4023],
│                   [ 0.3873, -1.5008, -1.9809],
│                   [ 0.7180, -1.4251, -0.4023],
│                   [ 0.3873, -1.5008, -1.9809],
│                   [ 0.7180, -1.4251, -0.4023],
│                   [ 0.3873, -1.5008, -1.9809],
│                   [ 0.7180, -1.4251, -0.4023],
│                   [ 0.3873, -1.5008, -1.9809]])
└── 'x' --> <FastTreeValue 0x7f8f8c899340>
    └── 'c' --> tensor([[ 3.9525, -0.4756, -0.1471, -1.0205],
                        [ 0.4388,  1.5708, -0.4817, -0.0550],
                 

In [23]:
%timeit t_cat(trees)

44.4 µs ± 697 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

85.1 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 3.9525, -0.4756, -0.1471, -1.0205],
                       [ 0.4388,  1.5708, -0.4817, -0.0550],
                       [-0.6711,  1.0422,  0.8273, -1.5164]],
              
                      [[ 3.9525, -0.4756, -0.1471, -1.0205],
                       [ 0.4388,  1.5708, -0.4817, -0.0550],
                       [-0.6711,  1.0422,  0.8273, -1.5164]],
              
                      [[ 3.9525, -0.4756, -0.1471, -1.0205],
                       [ 0.4388,  1.5708, -0.4817, -0.0550],
                       [-0.6711,  1.0422,  0.8273, -1.5164]],
              
                      [[ 3.9525, -0.4756, -0.1471, -1.0205],
                       [ 0.4388,  1.5708, -0.4817, -0.0550],
                       [-0.6711,  1.0422,  0.8273, -1.5164]],
              
                      [[ 3.9525, -0.4756, -0.1471, -1.0205],
                       [ 0.4388,  1.5708, -0.4817, -0.0550],
                       [-0.6711,  1.0422,  0.8273, -1.5164]],

In [26]:
%timeit Batch.stack(batches)

107 µs ± 1.09 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 3.9525, -0.4756, -0.1471, -1.0205],
                      [ 0.4388,  1.5708, -0.4817, -0.0550],
                      [-0.6711,  1.0422,  0.8273, -1.5164],
                      [ 3.9525, -0.4756, -0.1471, -1.0205],
                      [ 0.4388,  1.5708, -0.4817, -0.0550],
                      [-0.6711,  1.0422,  0.8273, -1.5164],
                      [ 3.9525, -0.4756, -0.1471, -1.0205],
                      [ 0.4388,  1.5708, -0.4817, -0.0550],
                      [-0.6711,  1.0422,  0.8273, -1.5164],
                      [ 3.9525, -0.4756, -0.1471, -1.0205],
                      [ 0.4388,  1.5708, -0.4817, -0.0550],
                      [-0.6711,  1.0422,  0.8273, -1.5164],
                      [ 3.9525, -0.4756, -0.1471, -1.0205],
                      [ 0.4388,  1.5708, -0.4817, -0.0550],
                      [-0.6711,  1.0422,  0.8273, -1.5164],
                      [ 3.9525, -0.4756, -0.1471, -1.0205],
                   

In [28]:
%timeit Batch.cat(batches)

261 µs ± 3.76 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

608 µs ± 26.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
